In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['hashes.txt', 'database.sqlite', 'Reviews.csv']


In [2]:
import keras
import tensorflow as tf
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,CuDNNLSTM,LSTM,Embedding,Dropout


In [4]:
df=pd.read_csv('../input/Reviews.csv')

In [5]:
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [6]:
df.isnull().sum()

Id                         0
ProductId                  0
UserId                     0
ProfileName               16
HelpfulnessNumerator       0
HelpfulnessDenominator     0
Score                      0
Time                       0
Summary                   27
Text                       0
dtype: int64

In [7]:
df.describe()

,Id,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time
count,568454.000000,568454.000000,568454.00000,568454.000000,5.684540e+05
mean,284227.500000,1.743817,2.22881,4.183199,1.296257e+09
std,164098.679298,7.636513,8.28974,1.310436,4.804331e+07
min,1.000000,0.000000,0.00000,1.000000,9.393408e+08
25%,142114.250000,0.000000,0.00000,4.000000,1.271290e+09
50%,284227.500000,0.000000,1.00000,5.000000,1.311120e+09
75%,426340.750000,2.000000,2.00000,5.000000,1.332720e+09
max,568454.000000,866.000000,923.00000,5.000000,1.351210e+09


In [8]:
df['Sentiment']=[1 if x>3 else 0 for x in df['Score']]

In [9]:
df.head(7)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Sentiment
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...,1
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,0
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...,1
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,0
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...,1
5,6,B006K2ZZ7K,ADT0SRK1MGOEU,Twoapennything,0,0,4,1342051200,Nice Taffy,I got a wild hair for taffy and ordered this f...,1
6,7,B006K2ZZ7K,A1SP2KVKFXXRU1,David C. Sullivan,0,0,5,1340150400,Great! Just as good as the expensive brands!,This saltwater taffy had great flavors and was...,1


In [10]:

import re
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
corpus=[]
ps=SnowballStemmer('english')
for i in range(df.shape[0]):
    review=re.sub('[^a-zA-Z]',' ',df['Text'][i])
    review=review.lower()
    review=review.split()
    review=[ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review=' '.join(review)
    
    corpus.append(review)

In [11]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                                   lower=True,split=' ')
tokenizer.fit_on_texts(corpus)

In [12]:
X = tokenizer.texts_to_sequences(corpus)

In [13]:
vocab_size=len(tokenizer.word_index)
print(vocab_size)
vocab_size=vocab_size+1

79643


In [14]:
X

[[65,
  204,
  2608,
  174,
  18,
  14,
  6,
  59,
  5,
  106,
  6,
  46,
  2,
  1615,
  495,
  298,
  108,
  37,
  3528,
  1447,
  821,
  6,
  37],
 [6,
  219,
  406,
  4063,
  122,
  211,
  211,
  138,
  120,
  115,
  2734,
  131,
  2383,
  1126,
  1396,
  2765,
  6,
  4063],
 [4022,
  180,
  4009,
  224,
  18262,
  1616,
  2100,
  274,
  206,
  10337,
  374,
  681,
  1101,
  4376,
  499,
  246,
  51,
  681,
  343,
  1109,
  561,
  4,
  58,
  47,
  441,
  39,
  1753,
  1612,
  709,
  13837,
  5085,
  8562,
  12860,
  39,
  10026,
  16404,
  380,
  1592,
  1235,
  8562],
 [46,
  1849,
  80,
  15129,
  385,
  59,
  89,
  367,
  1088,
  1124,
  780,
  25,
  5,
  62,
  582,
  460,
  4,
  1121],
 [12, 2655, 12, 28, 1409, 1342, 441, 2655, 526, 163, 2655, 740, 265],
 [89,
  977,
  398,
  2655,
  25,
  596,
  359,
  27,
  2655,
  69,
  87,
  4,
  1952,
  1088,
  1124,
  4601,
  1007,
  1159,
  362,
  727,
  74,
  24,
  347,
  244,
  694,
  4,
  239,
  473,
  79,
  217,
  277,
  134,
  68,
  

In [15]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
X = pad_sequences(X,maxlen=400,truncating='post')

In [16]:
X.shape

(568454, 400)

In [17]:
embed_dim = 450
lstm_out = 256
batch_size = 32

model = Sequential()
model.add(Embedding(vocab_size, embed_dim,input_length = X.shape[1]))
# model.add(LSTM(lstm_out,dropout=0.2,recurrent_dropout = 0.2,return_sequences=True))
# model.add(LSTM(lstm_out, dropout= 0.2, recurrent_dropout = 0.2))
model.add(CuDNNLSTM(lstm_out,return_sequences=True))
model.add(Dropout(0.2))
model.add(CuDNNLSTM(lstm_out))
model.add(Dropout(0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 400, 450)          35839800  
_________________________________________________________________
cu_dnnlstm (CuDNNLSTM)       (None, 400, 256)          724992    
_________________________________________________________________
dropout (Dropout)            (None, 400, 256)          0         
_________________________________________________________________
cu_dnnlstm_1 (CuDNNLSTM)     (None, 256)               526336    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 2)                 514       
Total params: 37,091,642
Trainable params: 37,091,642
Non-trainable params: 0
________________________________________________________________

In [18]:
Y = pd.get_dummies(df['Sentiment']).values

In [19]:
Y

array([[0, 1],
       [1, 0],
       [0, 1],
       ...,
       [0, 1],
       [0, 1],
       [0, 1]], dtype=uint8)

In [20]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(454763, 400) (454763, 2)
(113691, 400) (113691, 2)


In [21]:
model.fit(X_train, Y_train, batch_size =batch_size, nb_epoch = 1,validation_data=(X_test,Y_test))

/opt/conda/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/opt/conda/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 454763 samples, validate on 113691 samples
Epoch 1/1
381440/454763 [========================>.....] - ETA: 2:53 - loss: 0.2699 - acc: 0.8896

In [22]:
model.save('amazon.h5')

In [23]:
text='the service was decent but could have been better'

In [24]:
    review=re.sub('[^a-zA-Z]',' ',text)
    review=review.lower()
    review=review.split()
    review=[ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review=' '.join(review)

In [25]:
review

'servic decent could better'

In [26]:
x = tokenizer.texts_to_sequences([review])

In [27]:
x

[[531, 666, 81, 37]]

In [28]:
x = pad_sequences(x,maxlen=400,truncating='post')

In [29]:
x

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0

In [30]:
pred=model.predict(x)

In [31]:
pred

array([[0.5541994 , 0.44580057]], dtype=float32)

In [32]:
pred=np.argmax(pred,axis=1)

In [33]:
if(pred==1):
    print('Postive review Detected')
else:
    print('Negative review Detected')

Negative review Detected
